In [1]:
import os
import tempfile
import shutil
import subprocess
from tqdm import tqdm
from PIL import Image
from io import BytesIO
import numpy as np
from rembg import remove
import cv2

In [2]:
# Function to check if CUDA is available
def is_cuda_available() -> bool:
    result = subprocess.run(['ffmpeg', '-hwaccels'], capture_output=True, text=True)
    return 'cuda' in result.stdout

In [3]:
# Function to extract frames from a video using FFmpeg
def extract_frames(video_path: str, output_directory: str, fps: int) -> bool:
    os.makedirs(output_directory, exist_ok=True)
    command = [
        'ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', video_path,
        '-vf', f'fps={fps}', os.path.join(output_directory, 'frame_%04d.png')
    ]
    result = subprocess.run(command)
    return result.returncode == 0

In [4]:
# Function to remove the background from an image
def remove_background(image_path: str) -> Image.Image:
    with open(image_path, 'rb') as img_file:
        input_image = img_file.read()
    output_image = remove(input_image)
    return Image.open(BytesIO(output_image))

In [5]:
# Function to overlay foreground image on background image
def overlay_images(fg_image: Image.Image, bg_image: Image.Image) -> Image.Image:
    fg_image = fg_image.convert("RGBA")
    bg_image = bg_image.convert("RGBA")
    # Resize the foreground to match the background
    fg_image = fg_image.resize(bg_image.size, Image.LANCZOS)
    combined = Image.alpha_composite(bg_image, fg_image)
    return combined

In [6]:
# Create temporary directories
temp_dirs = [tempfile.mkdtemp() for _ in range(4)]
fg_frames_dir, bg_frames_dir, masked_dir, processed_frames_dir = temp_dirs

In [7]:
# Paths to input videos
fg_video = "E:\\My projects\\Dynamic-Rembg-music\\inputs\\foreground_video.mp4"  # Placeholder, update with actual name
bg_video = "E:\\My projects\\Dynamic-Rembg-music\\inputs\\background_video.mp4"

In [9]:
# Check if CUDA is available and set the flag
use_cuda = is_cuda_available()
print(use_cuda)

True


In [10]:
# Extract frames
fps = 24  # Placeholder, update with actual fps if needed
print("Extracting foreground frames...")
extract_frames(fg_video, fg_frames_dir, fps)
print("Extracting background frames...")
extract_frames(bg_video, bg_frames_dir, fps)

Extracting foreground frames...
Extracting background frames...


True

In [11]:
# Process foreground frames to remove background
fg_frame_files = sorted(os.listdir(fg_frames_dir))
print("Processing foreground frames to remove background...")
for frame_file in tqdm(fg_frame_files, desc="Processing foreground frames"):
    fg_frame_path = os.path.join(fg_frames_dir, frame_file)
    masked_image = remove_background(fg_frame_path)
    masked_image.save(os.path.join(masked_dir, frame_file))

Processing foreground frames to remove background...


Processing foreground frames: 100%|██████████| 241/241 [03:13<00:00,  1.24it/s]


In [12]:
# Combine foreground and background frames
bg_frame_files = sorted(os.listdir(bg_frames_dir))
print("Combining frames...")
for fg_frame_file, bg_frame_file in tqdm(zip(fg_frame_files, bg_frame_files), desc="Combining frames", total=len(fg_frame_files)):
    fg_frame_path = os.path.join(masked_dir, fg_frame_file)
    bg_frame_path = os.path.join(bg_frames_dir, bg_frame_file)
    fg_image = Image.open(fg_frame_path)
    bg_image = Image.open(bg_frame_path)
    combined_image = overlay_images(fg_image, bg_image)
    combined_image.save(os.path.join(processed_frames_dir, fg_frame_file))

Combining frames...


Combining frames: 100%|█████████▉| 240/241 [02:09<00:00,  1.85it/s]


In [13]:
# Merge processed frames into final video
output_video = 'output_video.mp4'  # Placeholder, update with desired output path
print("Merging processed frames into final video...")
command = [
    'ffmpeg', '-hide_banner', '-loglevel', 'error', '-framerate', str(fps),
    '-i', os.path.join(processed_frames_dir, 'frame_%04d.png'),
    '-c:v', 'libx264', '-pix_fmt', 'yuv420p', output_video
]
subprocess.run(command)

Merging processed frames into final video...


CompletedProcess(args=['ffmpeg', '-hide_banner', '-loglevel', 'error', '-framerate', '24', '-i', 'C:\\Users\\SAIDUL~1\\AppData\\Local\\Temp\\tmphvnowp6p\\frame_%04d.png', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', 'output_video.mp4'], returncode=0)

In [14]:
# Extract audio from the foreground video
audio_path = 'foreground_audio.aac'
print("Extracting audio from the foreground video...")
command = [
    'ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', fg_video,
    '-vn', '-acodec', 'aac', '-y', audio_path
]
subprocess.run(command)

Extracting audio from the foreground video...


CompletedProcess(args=['ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', 'E:\\My projects\\Dynamic-Rembg-music\\inputs - Copy\\foreground_video.mp4', '-vn', '-acodec', 'aac', '-y', 'foreground_audio.aac'], returncode=0)

In [15]:
# Merge audio with the final output video
final_output_video = 'final_output_video.mp4'
print("Merging audio with the final output video...")
command = [
    'ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', output_video, '-i', audio_path,
    '-c:v', 'copy', '-c:a', 'aac', '-map', '0:v:0', '-map', '1:a:0', '-y', final_output_video
]
subprocess.run(command)

Merging audio with the final output video...


CompletedProcess(args=['ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', 'output_video.mp4', '-i', 'foreground_audio.aac', '-c:v', 'copy', '-c:a', 'aac', '-map', '0:v:0', '-map', '1:a:0', '-y', 'final_output_video.mp4'], returncode=0)

In [16]:
# Cleanup temporary audio file and directories
os.remove(audio_path)
for temp_dir in temp_dirs:
    shutil.rmtree(temp_dir)

In [17]:
print("Process completed.")

Process completed.
